## 집 값 예측
- 예측할 변수 ['SalePrice']
- 평가: rmse, r2

    - rmse는 낮을 수록 좋은 성능
    - r2는 높을 수록 좋은 성능
   

In [42]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1168, 79), (292, 79), (1168, 2), (292, 2))

# 내가 풀기

# Data Load & Simple EDA

In [43]:
import pandas as pd
pd.set_option("display.max_columns", None)
y_train.head(1) # target

,Id,SalePrice
81,82,153500


In [44]:
X_train.isnull().sum().sort_values(ascending=False)

PoolQC           1163
MiscFeature      1124
Alley            1098
Fence             937
FireplaceQu       553
                 ... 
Heating             0
MSZoning            0
CentralAir          0
1stFlrSF            0
SaleCondition       0
Length: 79, dtype: int64

In [45]:
X_test.isnull().sum().sort_values(ascending=False)

PoolQC           290
MiscFeature      282
Alley            271
Fence            242
FireplaceQu      137
                ... 
Heating            0
MSZoning           0
CentralAir         0
Electrical         0
SaleCondition      0
Length: 79, dtype: int64

# Processing

In [46]:
# X_train.info()

In [47]:
X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])
target = y_train['SalePrice']

from sklearn.impute import SimpleImputer

imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

In [48]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size = 0.2, random_state=11)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

In [49]:
from sklearn.metrics import mean_squared_error, r2_score

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

print(r2_score(y_val, pred))
print(rmse(y_val, pred))

0.9015894023076253
23093.755181156972


In [50]:
pred = rf.predict(X_test)

submit = pd.DataFrame({'ID':y_test.Id, 'pred': pred}).reset_index(drop=True)
# submit.to_csv("12345.csv", index=False)
submit

,ID,pred
0,1381,87331.03
1,521,110535.00
2,1176,326486.03
3,352,198781.40
4,1336,165569.96
...,...,...
287,1284,149999.42
288,1040,87858.83
289,62,101372.90
290,1396,320083.07


# 풀이

# Data Load & Simple EDA

In [51]:
import pandas as pd

In [52]:
X_train.shape, X_test.shape

((1168, 36), (292, 36))

In [53]:
pd.set_option("display.max_columns", 100)
display(X_train.head(3))
display(X_test.head(3))

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
y_train['SalePrice'].hist()

In [ ]:
y_test['SalePrice'].hist()

In [ ]:
X_train.isnull().sum().sort_values(ascending=False)[:20]

In [ ]:
X_test.isnull().sum().sort_values(ascending=False)[:20]

In [ ]:
X_train.info()

# Preprocessing

In [ ]:
X_train = X_train.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object'])
target = y_train['SalePrice']

In [ ]:
X_train.head(3)

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

In [ ]:
X_train

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, target, test_size=0.15, random_state=2022)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

# Model

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor()
model.fit(X_tr, y_tr, verbose=False)
pred = model.predict(X_val)

print("R2 : " + str(r2_score(y_val, pred)))
print("RMSE : " + str(rmse(y_val, pred)))

In [ ]:
model = RandomForestRegressor()
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

print("R2 : " + str(r2_score(y_val, pred)))
print("RMSE : " + str(rmse(y_val, pred)))

In [ ]:
y = y_train['SalePrice']

In [ ]:
final_model = XGBRegressor()
final_model.fit(X_train, y)

prediction = final_model.predict(X_test)

## Prediction & to CSV

In [ ]:
submission = pd.DataFrame(data={
    'Id': y_test.Id,
    'income' : prediction
})

In [ ]:
submission.head()

In [ ]:
submission.to_csv("12345.csv", index=False)

# 결과 체점

In [ ]:
pred = model.predict(X_test)
print("RMSE : " + str(rmse(y_test['SalePrice'], prediction)))
print("R2 : " + str(r2_score(y_test['SalePrice'], prediction)))